# ליבת סמנטיקה

בדוגמת קוד זו, תשתמשו במסגרת ה-AI [Semantic Kernel](https://aka.ms/ai-agents-beginners/semantic-kernel) כדי ליצור סוכן בסיסי.

מטרת הדוגמה הזו היא להציג את השלבים שבהם נשתמש בהמשך בדוגמאות קוד נוספות בעת יישום דפוסי סוכנות שונים.


## ייבוא חבילות הפייתון הנדרשות


In [ ]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## יצירת הלקוח

בדוגמה זו, נשתמש ב-[GitHub Models](https://aka.ms/ai-agents-beginners/github-models) לצורך גישה ל-LLM.

ה-`ai_model_id` מוגדר כ-`gpt-4o-mini`. נסו לשנות את המודל למודל אחר הזמין בשוק של GitHub Models כדי לראות תוצאות שונות.

כדי שנוכל להשתמש ב-`Azure Inference SDK` שמשמש עבור ה-`base_url` של GitHub Models, נשתמש במחבר `OpenAIChatCompletion` בתוך Semantic Kernel. ישנם גם [מחברים זמינים נוספים](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) לשימוש ב-Semantic Kernel עבור ספקי מודלים אחרים.


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## יצירת הסוכן

כאן אנו יוצרים את הסוכן שנקרא `TravelAgent`.

בדוגמה זו, אנו משתמשים בהוראות פשוטות מאוד. ניתן לשנות את ההוראות הללו כדי לראות כיצד הסוכן מגיב בצורה שונה.


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## הפעלת הסוכן

כעת נוכל להפעיל את הסוכן על ידי הגדרת שרשור מסוג `ChatHistoryAgentThread`. כל הודעות מערכת נדרשות מסופקות כארגומנט `messages` במילת המפתח של `invoke_stream` של הסוכן.

לאחר שהגדרנו זאת, אנו יוצרים `user_inputs`, שיהיה מה שהמשתמש שולח לסוכן. במקרה זה, הגדרנו את ההודעה ל-`Plan me a sunny vacation`.

אתם מוזמנים לשנות את ההודעה כדי לראות כיצד הסוכן מגיב באופן שונה.


In [ ]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a day trip.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()


---

**כתב ויתור**:  
מסמך זה תורגם באמצעות שירות תרגום מבוסס בינה מלאכותית [Co-op Translator](https://github.com/Azure/co-op-translator). למרות שאנו שואפים לדיוק, יש לקחת בחשבון שתרגומים אוטומטיים עשויים להכיל שגיאות או אי דיוקים. המסמך המקורי בשפתו המקורית צריך להיחשב כמקור סמכותי. עבור מידע קריטי, מומלץ להשתמש בתרגום מקצועי על ידי אדם. איננו נושאים באחריות לאי הבנות או לפרשנויות שגויות הנובעות משימוש בתרגום זה.
